In [ ]:
import numpy as np
import pandas as pd
import os


In [ ]:
# 상수 정의
WINDOW_SIZE_20 = 20
WINDOW_SIZE_60 = 60
INPUT_FILE = 'feature_database_0628_ver.xlsx'
OUTPUT_FILE = 'Feature_data_base_20_60_final_06_28_1.xlsx'

def calculate_trend_slope(data, window_size):
    """단일 데이터 윈도우에 대한 트렌드 기울기를 계산합니다."""
    if len(data) < window_size:
        raise ValueError("데이터 길이가 구간 크기보다 작습니다.")
    
    window_data = data.astype(float)
    X = np.arange(window_size)
    
    nan_indices = np.isnan(window_data)
    if np.any(nan_indices):
        window_data = window_data[~nan_indices]
        X = X[~nan_indices]
    
    coefficients = np.polyfit(X, window_data, 1)
    slope = coefficients[0]
    
    abs_mean = np.mean(np.abs(window_data))
    if abs_mean == 0:
        return None
    else:
        return slope / abs_mean

def calculate_trend_slope_daily(data, window_size):
    """전체 데이터셋에 대해 일별로 트렌드 기울기를 계산합니다."""
    result = {}
    column_names = data.columns.tolist()
    
    for column in column_names:
        trend_slope_result = []
        for i in range(len(data) - window_size + 1):
            window_data = data[column].iloc[i:i+window_size]
            trend_slope = calculate_trend_slope(window_data, window_size)
            trend_slope_result.append(trend_slope)
        result[column] = trend_slope_result
    
    return result


In [ ]:
def load_and_preprocess_data(file_path):
    """데이터를 로드하고 전처리합니다."""
    try:
        df = pd.read_excel(file_path, sheet_name='Sheet1', index_col=0)
        df.index = df.index.strftime('%Y-%m-%d')
        df = df.dropna(axis=0)
        return df
    except Exception as e:
        print(f"데이터 로드 중 오류 발생: {e}")
        return None

def calculate_and_merge_slopes(df, window_size):
    """주어진 윈도우 크기로 트렌드 기울기를 계산하고 원본 데이터와 병합합니다."""
    result = calculate_trend_slope_daily(df, window_size)
    new_df = df.iloc[window_size-1:]
    result_df = pd.DataFrame(result)
    result_df.index = new_df.index
    return result_df

def save_results(df, file_path):
    """결과를 Excel 파일로 저장합니다."""
    try:
        df.to_excel(file_path, index=True)
        print(f"결과가 {file_path}에 저장되었습니다.")
    except Exception as e:
        print(f"파일 저장 중 오류 발생: {e}")



In [ ]:
def main():
    # 데이터 로드 및 전처리
    df = load_and_preprocess_data(INPUT_FILE)
    if df is None:
        return

    # 20일 및 60일 윈도우에 대한 트렌드 기울기 계산
    result_df_20 = calculate_and_merge_slopes(df, WINDOW_SIZE_20)
    result_df_60 = calculate_and_merge_slopes(df, WINDOW_SIZE_60)

    # 결과 병합
    merge_inner = pd.merge(df, result_df_20.add_suffix('_20'), left_index=True, right_index=True)
    merge_inner = pd.merge(merge_inner, result_df_60.add_suffix('_60'), left_index=True, right_index=True)

    # 인덱스를 datetime 형식으로 변환
    merge_inner.index = pd.to_datetime(merge_inner.index)

    # 월별 첫 번째 데이터 선택
    df_first_day_of_month = merge_inner.groupby(merge_inner.index.to_period('M')).first()
    df_first_day_of_month.index = df_first_day_of_month.index.strftime('%Y-%m-%d')

    # 결과 저장
    save_results(df_first_day_of_month, OUTPUT_FILE)

if __name__ == "__main__":
    main()

# 사용되지 않는 키워드 리스트 (필요시 활용)
keywords = ["auto","construct","capital_market","chemicals","equipment",
            "transport","semi","bank","steel",
            "telecom","staples","discretionary","kospi"]